In [1]:
import torch
from pyqcu.ascend import dslash
from pyqcu.ascend.include import *
from pyqcu.ascend import inverse
dof = 4
# latt_size = (16, 16, 16, 16)
latt_size = (32, 32, 32, 32)
# latt_size = (32, 32, 32, 64)
# latt_size = (4, 8, 8, 8)
# latt_size = (8, 8, 8, 4)
# latt_size = (16, 8, 8, 8)
# latt_size = (8, 8, 8, 8)
# latt_size = (4, 4, 4, 4)
kappa = 0.125
# dtype = torch.complex128
dtype = torch.complex64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Initialize lattice gauge theory
wilson = dslash.wilson_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=True
)
clover = dslash.clover_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
U = wilson.generate_gauge_field(sigma=0.1, seed=42)

wilson.check_su3(U)

null_vecs = torch.randn(dof, 4, 3, latt_size[3], latt_size[2], latt_size[1], latt_size[0],
                        dtype=dtype, device=device)
clover_term = clover.make_clover(U=U)
clover_term = clover.add_I(clover=clover_term)  # needed!

b = torch.randn_like(null_vecs[0])
U_eo = xxxtzyx2pxxxtzyx(input_array=U)
clover_eo = xxxtzyx2pxxxtzyx(input_array=clover_term)


    @@@@@@######QCU NOTES START######@@@@@@@
    Guide:
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support (when '#define _B

In [ ]:
from pyqcu.ascend import inverse
mg = inverse.mg(b=b, wilson=wilson, U_eo=U_eo,
                clover=clover, clover_eo=clover_eo, dof=dof,verbose=True)

Building grid list:
  Level 0: 32x32x32x32
  Level 1: 16x16x16x16
  Level 2: 8x8x8x8
  Level 3: 4x4x4x4
  Level 4: 2x2x2x2
self.grid_list:[[32, 32, 32, 32], [16, 16, 16, 16], [8, 8, 8, 8], [4, 4, 4, 4], [2, 2, 2, 2]]
Applying Dirac operator in eo...
  Source shape: torch.Size([4, 3, 32, 32, 32, 16])
  Gauge field shape: torch.Size([2, 3, 3, 4, 32, 32, 32, 16])
  Source norm: 0.7070584297180176
  Processing x_p-direction (axis=-1)...
    Hopping term norm: 1.4141170978546143
  Processing y-direction (axis=-2)...
    Hopping term norm: 1.4141169786453247
  Processing z-direction (axis=-3)...
    Hopping term norm: 1.4141169786453247
  Processing t-direction (axis=-4)...
    Hopping term norm: 1.4141169786453247
Dirac operator application complete in eo
  Dest_e norm: 0.3533904552459717
Applying Dirac operator in oe...
  Source shape: torch.Size([4, 3, 32, 32, 32, 16])
  Gauge field shape: torch.Size([2, 3, 3, 4, 32, 32, 32, 16])
  Source norm: 0.7071551084518433
  Processing x_p-directio

In [ ]:
x=mg.solve()